In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models.widgets import Select
from bokeh.plotting import figure, output_file, show, output_notebook

In [2]:
df = pd.read_csv('listings.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,65,2,182,2021-03-28,2.38,1,0
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85373,-87.69540,Entire home/apt,113,2,395,2020-07-14,2.67,1,180
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90166,-87.68021,Entire home/apt,65,2,394,2021-04-11,2.74,1,267
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91943,-87.63898,Entire home/apt,120,5,54,2021-01-15,0.63,11,1
4,10945,The Biddle House (#1),33004,At Home Inn,NaN,Lincoln Park,41.91196,-87.63981,Entire home/apt,175,4,22,2021-03-25,0.26,11,125


In [3]:
df.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                          169
neighbourhood_group               6386
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       1218
reviews_per_month                 1218
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6386 entries, 0 to 6385
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6386 non-null   int64  
 1   name                            6386 non-null   object 
 2   host_id                         6386 non-null   int64  
 3   host_name                       6217 non-null   object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   6386 non-null   object 
 6   latitude                        6386 non-null   float64
 7   longitude                       6386 non-null   float64
 8   room_type                       6386 non-null   object 
 9   price                           6386 non-null   int64  
 10  minimum_nights                  6386 non-null   int64  
 11  number_of_reviews               6386 non-null   int64  
 12  last_review                     51

In [5]:
df = df.drop('neighbourhood_group', axis = 1)
df = df.drop('last_review',axis = 1)
df = df.drop("reviews_per_month", axis = 1)
df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,65,2,182,1,0
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85373,-87.69540,Entire home/apt,113,2,395,1,180
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90166,-87.68021,Entire home/apt,65,2,394,1,267
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91943,-87.63898,Entire home/apt,120,5,54,11,1
4,10945,The Biddle House (#1),33004,At Home Inn,Lincoln Park,41.91196,-87.63981,Entire home/apt,175,4,22,11,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6381,49259148,Cozy apartment for you | 2BR in Chicago,359234447,Chelsia,Near North Side,41.89843,-87.63834,Entire home/apt,135,33,0,36,289
6382,49259244,Luxury homes,193776367,Joyce,Washington Heights,41.71982,-87.65894,Entire home/apt,319,1,0,1,175
6383,49264091,Modern 2 BR heaven on Michigan w optional parking,370154553,Innesa,Near South Side,41.85376,-87.62391,Entire home/apt,168,2,0,11,363
6384,49273283,No.1 Location Riverwalk/Michigan Ave,390856191,Nora,Near North Side,41.88948,-87.62600,Entire home/apt,120,84,0,5,170


"neighbourhood_group" column is totally null and no need of "last_review" and "reviews_per_month" in this tasks so i drop it from dataframe

In [6]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
count,6.386000e+03,6.386000e+03,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000
mean,3.056454e+07,1.150096e+08,41.898929,-87.663195,161.330254,9.024272,42.291889,15.982148,157.044785
std,1.407447e+07,1.144597e+08,0.058699,0.042860,360.998390,24.236983,69.484109,44.453948,136.275636
min,2.384000e+03,2.153000e+03,41.650640,-87.934820,0.000000,1.000000,0.000000,1.000000,0.000000
25%,1.998291e+07,1.969191e+07,41.873713,-87.686780,67.000000,1.000000,1.000000,1.000000,11.000000
50%,3.344259e+07,6.905864e+07,41.900755,-87.659125,102.000000,2.000000,12.000000,2.000000,126.000000
75%,4.293678e+07,1.846707e+08,41.939615,-87.632288,164.000000,4.000000,54.000000,9.000000,298.000000
max,4.927557e+07,3.967887e+08,42.022200,-87.535880,9999.000000,500.000000,801.000000,233.000000,365.000000


In [7]:
df_q1 = df.groupby(by = 'neighbourhood', as_index = False)['calculated_host_listings_count'].sum()
df_q1.head()

,neighbourhood,calculated_host_listings_count
0,Albany Park,164
1,Archer Heights,8
2,Armour Square,545
3,Ashburn,25
4,Auburn Gresham,11


In [8]:
nebur_data = gpd.read_file('neighbourhoods.geojson')
nebur_data.head()

,neighbourhood,neighbourhood_group,geometry
0,Lincoln Square,None,"MULTIPOLYGON (((-87.67441 41.97610, -87.67440 ..."
1,Hyde Park,None,"MULTIPOLYGON (((-87.58038 41.80253, -87.58040 ..."
2,Woodlawn,None,"MULTIPOLYGON (((-87.57715 41.78615, -87.57715 ..."
3,Rogers Park,None,"MULTIPOLYGON (((-87.65456 41.99817, -87.65574 ..."
4,Jefferson Park,None,"MULTIPOLYGON (((-87.75263 41.96797, -87.75279 ..."


In [9]:
nb_data = nebur_data.drop("neighbourhood_group", axis = 1)
nb_data.head()

,neighbourhood,geometry
0,Lincoln Square,"MULTIPOLYGON (((-87.67441 41.97610, -87.67440 ..."
1,Hyde Park,"MULTIPOLYGON (((-87.58038 41.80253, -87.58040 ..."
2,Woodlawn,"MULTIPOLYGON (((-87.57715 41.78615, -87.57715 ..."
3,Rogers Park,"MULTIPOLYGON (((-87.65456 41.99817, -87.65574 ..."
4,Jefferson Park,"MULTIPOLYGON (((-87.75263 41.96797, -87.75279 ..."


### Mergeing data

In [10]:
data_q1 = df_q1.merge(nb_data, left_on = 'neighbourhood', right_on = 'neighbourhood')
data_q1

,neighbourhood,calculated_host_listings_count,geometry
0,Albany Park,164,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1,Archer Heights,8,"MULTIPOLYGON (((-87.71437 41.82604, -87.71436 ..."
2,Armour Square,545,"MULTIPOLYGON (((-87.62917 41.84556, -87.62947 ..."
3,Ashburn,25,"MULTIPOLYGON (((-87.71255 41.75734, -87.71252 ..."
4,Auburn Gresham,11,"MULTIPOLYGON (((-87.63990 41.75615, -87.63990 ..."
...,...,...,...
72,West Lawn,14,"MULTIPOLYGON (((-87.71255 41.75734, -87.71278 ..."
73,West Pullman,4,"MULTIPOLYGON (((-87.61828 41.65911, -87.61829 ..."
74,West Ridge,465,"MULTIPOLYGON (((-87.68465 42.01949, -87.68464 ..."
75,West Town,4781,"MULTIPOLYGON (((-87.65686 41.91078, -87.65685 ..."


In [11]:
data_q1.rename(columns = {'calculated_host_listings_count':'listings_count'}, inplace = True)

In [12]:
data_q1.head()

,neighbourhood,listings_count,geometry
0,Albany Park,164,"MULTIPOLYGON (((-87.70404 41.97355, -87.70403 ..."
1,Archer Heights,8,"MULTIPOLYGON (((-87.71437 41.82604, -87.71436 ..."
2,Armour Square,545,"MULTIPOLYGON (((-87.62917 41.84556, -87.62947 ..."
3,Ashburn,25,"MULTIPOLYGON (((-87.71255 41.75734, -87.71252 ..."
4,Auburn Gresham,11,"MULTIPOLYGON (((-87.63990 41.75615, -87.63990 ..."


In [13]:
data_q1 = data_q1.dropna()

In [14]:
data_q1.isnull().sum()

neighbourhood     0
listings_count    0
geometry          0
dtype: int64

In [15]:
data_q1 = gpd.GeoDataFrame(data_q1,geometry='geometry')
data_q1.crs

In [16]:
geosource = GeoJSONDataSource(geojson = data_q1.to_json())

In [17]:
# Define color palettes

palette = brewer['BuGn'][9]
palette = palette[::-1]  # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette)
# Define custom tick labels for color bar.
tick_labels = {'0': '0', '5000000': '5,000,000',
 '10000000':'10,000,000', '15000000':'15,000,000',
 '20000000':'20,000,000', '25000000':'25,000,000',
 '30000000':'30,000,000', '35000000':'35,000,000',
 '40000000':'40,000,000+'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)
output_notebook()
# Create figure object.

p = figure(title = 'Top areas with highest number of Listings', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field': 'listings_count', 'transform': color_mapper},
                   line_color = 'black', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# # Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Neighbourhood','@neighbourhood'),
                                ('Listings','@listings_count')]))
p.add_layout(color_bar, 'below')
show(p)

Loading BokehJS ...

In the above map its show the neighbourhood with the highest listing.
Near North side is the highest listing that is 33157

## Thematic map of the neighbourhood

In [18]:
from pyproj import Proj, transform

inProj = Proj('+init=epsg:3857')
outProj = Proj('+init=epsg:4326')

lons, lats = [],[]
for lon, lat in list(zip(df['longitude'], df['latitude'])):
    x, y = transform(outProj, inProj,lon,lat)
    lons.append(x)
    lats.append(y)
    

C:\Users\RAHUL\anaconda3\lib\site-packages\pyproj\crs\crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
C:\Users\RAHUL\anaconda3\lib\site-packages\pyproj\crs\crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
C:\Users\RAHUL\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [19]:
df["MercatorX"] = lons
df["MercatorY"] = lats
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,MercatorX,MercatorY
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,65,2,182,1,0,-9.750229e+06,5.129261e+06
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85373,-87.69540,Entire home/apt,113,2,395,1,180,-9.762207e+06,5.139094e+06
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90166,-87.68021,Entire home/apt,65,2,394,1,267,-9.760516e+06,5.146260e+06
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91943,-87.63898,Entire home/apt,120,5,54,11,1,-9.755927e+06,5.148918e+06
4,10945,The Biddle House (#1),33004,At Home Inn,Lincoln Park,41.91196,-87.63981,Entire home/apt,175,4,22,11,125,-9.756019e+06,5.147801e+06


In [23]:
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
from bokeh.io import show
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Legend, LegendItem

wikimedia = get_provider(WIKIMEDIA)

lon1, lat1 = transform(outProj,inProj,-87.58,41.62)
lon2, lat2 = transform(outProj,inProj,-87.52,42.025)
 

C:\Users\RAHUL\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
C:\Users\RAHUL\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


In [24]:
room_types = list(df['room_type'].unique())
room_types.sort()
room_types

['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

In [33]:
room_selection = CheckboxGroup(labels=room_types,active = [0, 1])

In [26]:
[room_selection.labels[i] for i in room_selection.active]

['Entire home/apt', 'Hotel room']

In [27]:
src = ColumnDataSource(df)
subj_list = sorted(list(set(src.data['room_type'])))

callback = CustomJS(args=dict(src=src), code='''
    src.change.emit();
''')

js_filter = CustomJSFilter(code='''
var indices = [];
for (var i = 0; i < src.get_length(); i++){
    if (src.data['room_type'][i] == select.value){
        indices.push(true);
    } else {
        indices.push(false);
    }
}
return indices;
''')

options = subj_list
select = Select(title='Select Room', value=options[0], options=options)

select.js_on_change('value', callback)

view = CDSView(source=src, filters=[js_filter])

wikimedia = get_provider(WIKIMEDIA)

lon1, lat1 = transform(outProj,inProj,-87.58,41.62)
lon2, lat2 = transform(outProj,inProj,-87.52,42.025)

p = figure(plot_width=1440, plot_height=720,
           x_range=(lon1, lon2), y_range=(lat1, lat2),
           x_axis_type="mercator", y_axis_type="mercator",
           tooltips=[
                    ("Neighbourhood", "@neighbourhood"), ("Name", "@host_name"), ("Room Type", "@room_type"),("Price","@price")
                    ],
           title="Airbnb hosts Chicago")

p.add_tile(wikimedia)


p.circle (x="MercatorX", y="MercatorY",
         size=5,
         fill_color="dodgerblue", line_color="dodgerblue",
         fill_alpha=0.3,
         source=src)

show(column(select, p))

C:\Users\RAHUL\anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
C:\Users\RAHUL\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


By using dropdown we can filter the types of room accordingly

## Top 10 hosts by their total listings

In [30]:
top_hosts = df.groupby(['host_id','host_name']).agg({'calculated_host_listings_count':'sum'}).reset_index()
top_hosts

,host_id,host_name,calculated_host_listings_count
0,2153,Linda,36
1,2613,Rebecca,1
2,4434,Kellen,25
3,5775,Craig & Kathleen,1
4,6162,Jackie,1
...,...,...,...
3293,395498979,Milan,1
3294,395849745,Derrick,1
3295,396515206,Frank,4
3296,396622927,Lilly,4


In [31]:
top_hosts.sort_values('calculated_host_listings_count', ascending=False).head(10)


,host_id,host_name,calculated_host_listings_count
2056,107434423,Blueground,54289
301,3965428,Rob,6241
1466,47172572,Zencity,4624
660,12243051,Sonder,2809
2454,170785489,Dmd,2601
549,9094538,Nicole,1849
3180,359234447,Chelsia,1296
1687,63313003,K,1156
1508,49626033,Roma,961
1205,33127842,Kari,841


### so these are the top 10 hosts with their host id and total number of listing.